Note: The following is originally from [machinelearningplus.com](https://www.machinelearningplus.com/statistics/mahalanobis-distance/)

# 0. Imports

In [1]:
import pandas as pd
import scipy as sp
import numpy as np

# 1. Data

In [2]:
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv'
df = pd.read_csv(filepath).iloc[:, [0, 4, 6]]
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


# 2. Function for calculation of MD

In [3]:
def mahalanobis(x=None, data=None, cov=None):
    """
    Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data, axis=0)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = sp.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()  # Can't understand why .diagonal() is used

In [4]:
df_x = df.head(500).copy()
df_x_test = df_x.copy()  # To check the step-by-step operation below
df_x['mahala'] = mahalanobis(x=df_x, data=df)
df_x.head()

,carat,depth,price,mahala
0,0.23,61.5,326,1.709860
1,0.21,59.8,326,3.540097
2,0.23,56.9,327,12.715021
3,0.29,62.4,334,1.454469
4,0.31,63.3,335,2.347239


#### Checking step by step operation of the mahalanobis function

In [5]:
x_minus_mu = df_x_test - np.mean(df, axis=0)

In [6]:
x_minus_mu

,carat,depth,price
0,-0.56794,-0.249405,-3606.799722
1,-0.58794,-1.949405,-3606.799722
2,-0.56794,-4.849405,-3605.799722
3,-0.50794,0.650595,-3598.799722
4,-0.48794,1.550595,-3597.799722
...,...,...,...
495,-0.08794,0.950595,-1110.799722
496,-0.08794,-0.449405,-1110.799722
497,-0.09794,-1.549405,-1110.799722
498,-0.09794,-1.049405,-1110.799722


In [7]:
cov = np.cov(df.values.T)

In [8]:
cov

array([[ 2.24686660e-01,  1.91665282e-02,  1.74276536e+03],
       [ 1.91665282e-02,  2.05240384e+00, -6.08537121e+01],
       [ 1.74276536e+03, -6.08537121e+01,  1.59156294e+07]])

In [9]:
cov.shape

(3, 3)

In [10]:
inv_covmat = sp.linalg.inv(cov)

In [11]:
inv_covmat

array([[ 2.98255446e+01, -3.75404629e-01, -3.26733989e-03],
       [-3.75404629e-01,  4.92013886e-01,  4.29881243e-05],
       [-3.26733989e-03,  4.29881243e-05,  4.20770213e-07]])

In [12]:
left_term = np.dot(x_minus_mu, inv_covmat)

In [13]:
left_term.shape

(500, 3)

In [14]:
mahal = np.dot(left_term, x_minus_mu.T)

In [15]:
mahal.shape

(500, 500)

# 3. Use Cases

## 3.1. Multivariate outlier detection using Mahalanobis distance

Assuming that the test statistic follows chi-square distributed with ‘n’ degree of freedom, the critical value at a 0.01 significance level and 2 degrees of freedom is computed as:

In [16]:
# Critical values for two degrees of freedom
from scipy.stats import chi2
chi2.ppf((1-0.01), df=2)

9.21034037197618

That mean an observation can be considered as extreme if its Mahalanobis distance exceeds 9.21. If you prefer P values instead to determine if an observation is extreme or not, the P values can be computed as follows:

In [17]:
# Compute the P-Values
df_x['p_value'] = 1 - chi2.cdf(df_x['mahala'], 2)

# Extreme values with a significance level of 0.01
df_x.loc[df_x.p_value < 0.01].head(10)

,carat,depth,price,mahala,p_value
2,0.23,56.9,327,12.715021,0.001734
91,0.86,55.1,2757,23.909643,0.000006
97,0.96,66.3,2759,11.781773,0.002765
172,1.17,60.2,2774,9.279459,0.009660
204,0.98,67.9,2777,20.086616,0.000043
221,0.70,57.2,2782,10.405659,0.005501
227,0.84,55.1,2782,23.548379,0.000008
255,1.05,65.8,2789,11.237146,0.003630
284,1.00,58.2,2795,10.349019,0.005659
298,1.01,67.4,2797,17.716144,0.000142


If you compare the above observations against rest of the dataset, they are clearly extreme.

## 3.2. Mahalanobis Distance for Classification Problems

Mahalanobis distance can be used for classification problems. A naive implementation of a Mahalanobis classifier is coded below. The intuition is that, an observation is assigned the class that it is closest to based on the Mahalanobis distance. Let’s see an example implementation on the BreastCancer dataset, where the objective is to determine if a tumour is benign or malignant.

In [18]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/selva86/datasets/master/BreastCancer.csv', 
    usecols=[
        'Cl.thickness', 'Cell.size', 'Marg.adhesion', 'Epith.c.size', 'Bare.nuclei',
        'Bl.cromatin', 'Normal.nucleoli', 'Mitoses', 'Class'
    ]
)

df.dropna(inplace=True)  # drop missing values.
df.head()

,Cl.thickness,Cell.size,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,5,1,1,2,1.0,3,1,1,0
1,5,4,5,7,10.0,3,2,1,0
2,3,1,1,2,2.0,3,1,1,0
3,6,8,1,3,4.0,3,7,1,0
4,4,1,3,2,1.0,3,1,1,0


Let’s split the dataset in 70:30 ratio as Train and Test. And the training dataset is split into homogeneous groups of ‘pos'(1) and ‘neg'(0) classes. To predict the class of the test dataset, we measure the Mahalanobis distances between a given observation (row) and both the positive (xtrain_pos) and negative datasets(xtrain_neg). Then that observation is assigned the class based on the group it is closest to.

In [19]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(
    df.drop('Class', axis=1), 
    df['Class'], 
    test_size=.3, 
    random_state=100
)

# Split the training data as pos and neg
xtrain_pos = xtrain.loc[ytrain == 1, :]
xtrain_neg = xtrain.loc[ytrain == 0, :]

Let’s build the `MahalanobiBinaryClassifier`. To do that, you need to define the `predict_proba()` and the `predict()` methods. This classifier does not require a separate `fit()` (training) method.

In [20]:
class MahalanobisBinaryClassifier():
    def __init__(self, xtrain, ytrain):
        self.xtrain_pos = xtrain.loc[ytrain == 1, :]
        self.xtrain_neg = xtrain.loc[ytrain == 0, :]

    def predict_proba(self, xtest):
        pos_neg_dists = [
            (p, n) for p, n in zip(
                mahalanobis(xtest, self.xtrain_pos), 
                mahalanobis(xtest, self.xtrain_neg)
            )
        ]
        return np.array([(1-n/(p+n), 1-p/(p+n)) for p, n in pos_neg_dists])

    def predict(self, xtest):
        return np.array([np.argmax(row) for row in self.predict_proba(xtest)])

In [21]:
clf = MahalanobisBinaryClassifier(xtrain, ytrain)
pred_probs = clf.predict_proba(xtest)
pred_class = clf.predict(xtest)

In [22]:
# Pred and Truth
pred_actuals = pd.DataFrame(
    [(pred, act) for pred, act in zip(pred_class, ytest)], 
    columns=['pred', 'true']
)
pred_actuals

,pred,true
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
200,1,1
201,0,0
202,0,0
203,0,0


Let’s see how the classifier performed on the test dataset.

In [23]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

truth = pred_actuals.loc[:, 'true']
pred = pred_actuals.loc[:, 'pred']
scores = pred_probs[:, 1]
print('AUROC: ', roc_auc_score(truth, scores))
print('\nConfusion Matrix: \n', confusion_matrix(truth, pred))
print('\nAccuracy Score: ', accuracy_score(truth, pred))
print('\nClassification Report: \n', classification_report(truth, pred))

AUROC:  0.990974358974359

Confusion Matrix: 
 [[113  17]
 [  0  75]]

Accuracy Score:  0.9170731707317074

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.87      0.93       130
           1       0.82      1.00      0.90        75

    accuracy                           0.92       205
   macro avg       0.91      0.93      0.91       205
weighted avg       0.93      0.92      0.92       205



## 3.3. One-Class Classification

**One Class classification is a type of algorithm where the training dataset contains observations belonging to only one class.**

With only that information known, the objective is to figure out if a given observation in a new (or test) dataset belongs to that class. You might wonder when would such a situation occur.

Well, it’s a quite common problem in Data Science.

For example consider the following situation: You have a large dataset containing millions of records that are NOT yet categorized as 1’s and 0’s. But you also have with you a small sample dataset containing only positive (1’s) records. By learning the information in this sample dataset, you want to classify all the records in the large dataset as 1’s and 0’s. Based on the information from the sample dataset, it is possible to tell if any given sample is a 1 or 0 by viewing only the 1’s (and having no knowledge of the 0’s at all). This can be done using Mahalanobis Distance.

Let’s try this on the BreastCancer dataset, only this time we will consider only the malignant observations (class column=1) in the training data.

In [24]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/selva86/datasets/master/BreastCancer.csv', 
    usecols=[
        'Cl.thickness', 'Cell.size', 'Marg.adhesion', 'Epith.c.size', 'Bare.nuclei', 
        'Bl.cromatin', 'Normal.nucleoli', 'Mitoses', 'Class'
    ]
)

df.dropna(inplace=True)  # drop missing values.

**Splitting 50% of the dataset into training and test. Only the 1’s are retained in the training data.**

In [25]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(
    df.drop('Class', axis=1), 
    df['Class'], 
    test_size=.5, 
    random_state=100
)

# Split the training data as pos and neg
xtrain_pos = xtrain.loc[ytrain == 1, :]

Let’s build the `MahalanobisOneClassClassifier` and get the mahalanobis distance of each datapoint in x from the training set (xtrain_pos).

In [26]:
class MahalanobisOneclassClassifier():
    def __init__(self, xtrain, significance_level=0.01):
        self.xtrain = xtrain
        self.critical_value = chi2.ppf((1-significance_level), df=xtrain.shape[1] - 1) # df = degree of freedom
        print('Critical value is: ', self.critical_value)

    def predict_proba(self, xtest):
        mahalanobis_dist = mahalanobis(xtest, self.xtrain)
        self.pvalues = 1 - chi2.cdf(mahalanobis_dist, 2)
        return mahalanobis_dist

    def predict(self, xtest):
        return np.array([int(i) for i in self.predict_proba(xtest) > self.critical_value])

In [27]:
clf = MahalanobisOneclassClassifier(xtrain_pos, significance_level=0.05)
mahalanobis_dist = clf.predict_proba(xtest)

Critical value is:  14.067140449340169


In [28]:
# Pred and Truth
mdist_actuals = pd.DataFrame(
    [(m, act) for m, act in zip(mahalanobis_dist, ytest)], 
    columns=['mahal', 'true_class']
)
mdist_actuals[:5]

,mahal,true_class
0,13.104716,0
1,14.408570,1
2,14.932236,0
3,14.588622,0
4,15.471064,0


We have the Mahalanobis distance and the actual class of each observation. I would expect those observations with low Mahalanobis distance to be 1’s. So, I sort the mdist_actuals by Mahalanobis distance and quantile cut the rows into 10 equal sized groups. The observations in the top quantiles should have more 1’s compared to the ones in the bottom. Let’s see.

In [29]:
# quantile cut in 10 pieces
mdist_actuals['quantile'] = pd.qcut(
    mdist_actuals['mahal'], 
    q=[0, .10, .20, .3, .4, .5, .6, .7, .8, .9, 1], 
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
)

# sort by mahalanobis distance
mdist_actuals.sort_values('mahal', inplace=True)
perc_truths = mdist_actuals.groupby('quantile').agg(
    {'mahal': np.mean, 'true_class': np.sum}
).rename(columns={'mahal': 'avg_mahaldist', 'true_class': 'sum_of_trueclass'})
perc_truths

,avg_mahaldist,sum_of_trueclass
quantile,,
1,3.765496,33
2,6.511026,32
3,9.272944,30
4,12.209504,20
5,14.455050,4
6,15.684493,4
7,17.368633,3
8,18.840714,1
9,21.533159,2


If you notice above, nearly 90% of the 1’s (malignant cases) fall within the first 40%ile of the Mahalanobis distance.

Incidentally, all of these are lower than the critical value pf 14.05. So, let’s the critical value as the cutoff and mark those observations with Mahalanobis distance less than the cutoff as positive.

In [30]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

# Positive if mahalanobis 
pred_actuals = pd.DataFrame(
    [(int(p), y) for y, p in zip(ytest, clf.predict_proba(xtest) < clf.critical_value)], 
    columns=['pred', 'true']
)

# Accuracy Metrics
truth = pred_actuals.loc[:, 'true']
pred = pred_actuals.loc[:, 'pred']
print('\nConfusion Matrix: \n', confusion_matrix(truth, pred))
print('\nAccuracy Score: ', accuracy_score(truth, pred))
print('\nClassification Report: \n', classification_report(truth, pred))


Confusion Matrix: 
 [[183  29]
 [ 15 115]]

Accuracy Score:  0.8713450292397661

Classification Report: 
               precision    recall  f1-score   support

           0       0.92      0.86      0.89       212
           1       0.80      0.88      0.84       130

    accuracy                           0.87       342
   macro avg       0.86      0.87      0.87       342
weighted avg       0.88      0.87      0.87       342



**So, without the knowledge of the benign class, we are able to accurately predict the class of 87% of the observations.**